In [1]:
import os
from pycocotools.coco import COCO
import csv, json
import pandas as pd
import cv2

In [2]:
instancesFile = '../annotations/instances_val2017.json'
annFile = '../annotations/captions_val2017.json'
cocoInstances = COCO(instancesFile)
cocoAnn = COCO(annFile)

loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [11]:
with open('../annotations/otherImgAnnotations.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['image', 'gt_caption'])
    for image in os.listdir('../images/other'):
        img_id = int(image.split('_')[3])
        annIds = cocoAnn.getAnnIds(imgIds = img_id)
        anns = cocoAnn.loadAnns(ids = annIds)
        for ann in anns:
            writer.writerow([image, ann])

In [6]:
df = pd.read_csv('../annotations/otherImgAnnotations.csv')
images = df["image"].tolist()
caption = df["pair_caption"].tolist()
visited = {}
output = {"annotations":[], "images":[]}
for i, im in enumerate(images):
    im_id = str(im.split("_")[3]) + str(im.split("_")[4])
    if im_id not in visited:
        output["images"].append({"id": int(im_id.split(".")[0]), "file_name": im_id})
        visited[im_id] = True
    output["annotations"].append({"image_id": int(im_id.split(".")[0]), "caption": caption[i]})

In [7]:
with open('../annotations/augAnn.json', 'w') as f:
    json.dump(output, f)

In [17]:
# copy and rename files
import os
import shutil

def copy_rename(old_file_name, new_file_name):
    src_dir = ("../images/aug/")
    dst_dir = ("../images/aug2/")
    src_file = os.path.join(src_dir, old_file_name)
    shutil.copy(src_file,dst_dir)
    dst_file = os.path.join(dst_dir, old_file_name)
    new_dst_file_name = os.path.join(dst_dir, new_file_name)
    os.rename(dst_file, new_dst_file_name)

In [4]:
for file in os.listdir("../images/aug"):
    filename = "../images/aug/" + file
    new_filename = "../images/aug3/" + file
    img = cv2.imread(filename)
    w, h, _ = img.shape

    scale_percent = 400 / w
    height = int(h * scale_percent)
    dsize = (height, 400)
    output = cv2.resize(img, dsize, interpolation = cv2.INTER_AREA)
    cv2.imwrite(new_filename, output) 